In [7]:
import nltk
from nltk import sent_tokenize 
from nltk import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import TweetTokenizer


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

#ML models, and other tools
import torch as torch
from torch import nn as nn




import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# X_train, X_test, Y_train, Y_test = train_test_split(
#     df['tweet_token'],
#     df['class'],
#     test_size = 0.2,
#     random_state = 10,
#     stratify=df['class'].values,
# )

In [9]:
#Creating glove to then use for overlapping with our dataset
import torchtext

#glove is a pre-trained library of vocabulary that has every word given thier corresponding identity number
glove = torchtext.vocab.GloVe(name='6B',dim=50)
glove.itos

['the',
 ',',
 '.',
 'of',
 'to',
 'and',
 'in',
 'a',
 '"',
 "'s",
 'for',
 '-',
 'that',
 'on',
 'is',
 'was',
 'said',
 'with',
 'he',
 'as',
 'it',
 'by',
 'at',
 '(',
 ')',
 'from',
 'his',
 "''",
 '``',
 'an',
 'be',
 'has',
 'are',
 'have',
 'but',
 'were',
 'not',
 'this',
 'who',
 'they',
 'had',
 'i',
 'which',
 'will',
 'their',
 ':',
 'or',
 'its',
 'one',
 'after',
 'new',
 'been',
 'also',
 'we',
 'would',
 'two',
 'more',
 "'",
 'first',
 'about',
 'up',
 'when',
 'year',
 'there',
 'all',
 '--',
 'out',
 'she',
 'other',
 'people',
 "n't",
 'her',
 'percent',
 'than',
 'over',
 'into',
 'last',
 'some',
 'government',
 'time',
 '$',
 'you',
 'years',
 'if',
 'no',
 'world',
 'can',
 'three',
 'do',
 ';',
 'president',
 'only',
 'state',
 'million',
 'could',
 'us',
 'most',
 '_',
 'against',
 'u.s.',
 'so',
 'them',
 'what',
 'him',
 'united',
 'during',
 'before',
 'may',
 'since',
 'many',
 'while',
 'where',
 'states',
 'because',
 'now',
 'city',
 'made',
 'like',
 

In [10]:
import pandas as pd
dataframe = pd.read_csv('./labeled_data.csv')
dataframe = dataframe[['class', 'tweet']]
dataframe = dataframe.replace([0,1,2], ['Hate Speech', 'Offensive Language', 'Other'])


dataframe = dataframe.apply(lambda x: x.astype(str).str.lower())
dataframe.tweet = dataframe.tweet.str.replace('[^\s\w]','')
dataframe.tweet = dataframe.tweet.str.replace('[^\s\w]','')
dataframe.tweet = dataframe.tweet.str.replace('rt', '')
# dataframe['tweet_recon'] = dataframe['tweet_token'].apply(lambda x: list(ps.stem(i) for i in x))
# dataframe['tweet_recon'] = dataframe['tweet_recon'].apply(lambda x: ' '.join(list(i for i in x if i not in stops)))
# tweet_tokenizer = Tokenizer(num_words = 4500, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ') 
# tweet_tokenizer.fit_on_texts(texts = dataframe['tweet_token'])

# dataframe['tweet_token'] = tweet_tokenizer.texts_to_sequences(texts = dataframe['tweet_token'])

dataframe.to_csv('./new_data.csv')

<ipython-input-10-98a6d4f1103e>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe.tweet = dataframe.tweet.str.replace('[^\s\w]','')
<ipython-input-10-98a6d4f1103e>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe.tweet = dataframe.tweet.str.replace('[^\s\w]','')


In [14]:
from collections import Counter
class Merger():
    
    def __init__(self, csv_path, max_length):
        self.dataframe = pd.read_csv(csv_path)
        self.max_length = max_length
        self.label_dict = {
            "hate speech" : 0,
            "offensive language" : 1,
            "other": 2
        }
        self.dataframe['tweet_token'] = self.dataframe['tweet'].apply(lambda x: word_tokenize(x))
        
        all_mentioned_words = []
        for words in self.dataframe['tweet_token']:
            all_mentioned_words += words
        frequency = Counter(all_mentioned_words)
        self.vocab = torchtext.vocab.Vocab(counter = frequency, min_freq = 25, vectors = glove)
        
        
#         print(self.dataframe.head())

    def __len__(self):
        return len(self.dataframe['tweet'])
    
    def back_to_text(self, tokens):
        text = ''
        for tok in tokens:
            text += self.vocab.itos[tok] + " "
        return text
    
    
    def __getitem__(self, index):
        label = self.label_dict[self.dataframe['class'][index]]
        label = torch.tensor(label)
        int_tokens = []
        tweet_tokens = self.dataframe['tweet_token'][index]
        for token in tweet_tokens:
            int_tokens.append(self.vocab[token])
        if(len(int_tokens) < self.max_length):
            num_to_pad = self.max_length - len(int_tokens)
            int_tokens += [0] * num_to_pad
        else:
            int_tokens = int_tokens[:self.max_length]
        int_tokens = torch.tensor(int_tokens)
        return(int_tokens, label)
        
            
        
    

In [77]:
df = pd.read_csv("new_data.csv")
df.head()

,Unnamed: 0,class,tweet
0,0,other,mayasolovely as a woman you shouldnt complai...
1,1,offensive language,mleew17 boy dats coldtyga dwn bad for cuffin...
2,2,offensive language,urkindofbrand dawg 80sbaby4life you ever fu...
3,3,offensive language,c_g_anderson viva_based she look like a tranny
4,4,offensive language,shenikarobes the shit you hear about me migh...


In [78]:
class_counter = Counter(df['class'])
class_counter

Counter({'other': 4163, 'offensive language': 19190, 'hate speech': 1430})

In [80]:
ratios = [x/24783 for x in class_counter.values()]
ratios

[0.16797804946939435, 0.7743211072105879, 0.05770084332001776]

In [81]:
ratios[1]/ratios[0], ratios[1]/ratios[2]


(4.6096564977179915, 13.419580419580418)

In [16]:
dataset = Merger( './new_data.csv', 50)
len(dataset)


24783

In [20]:
import random
indices_list = list(range(0, 24783))
train_amount = int(0.8* len(dataset))
train_indices = list(range(0, train_amount))
test_indices = list(range(train_amount, len(dataset)))

In [21]:
from torch.utils.data import Subset
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

In [22]:
from torch.utils.data import Dataset, DataLoader
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 32)
train_dataloader

In [23]:
batched_data, batched_labels = next(iter(train_dataloader))
batched_data.shape, batched_labels.shape

(torch.Size([32, 50]), torch.Size([32]))

In [110]:
class NLP_model(nn.Module):
    def __init__(self, vocab, emb_size, num_classes):
        super().__init__()
        self.num_words = len(vocab)
        self.emb_size = emb_size
        self.emb = nn.Embedding(self.num_words, self.emb_size)
        self.emb.from_pretrained(vocab.vectors)
        self.lstm = nn.LSTM(input_size = emb_size, hidden_size = 32, batch_first = True, num_layers = 2,bidirectional= True)
        self.relu = nn.ReLU()
        self.lin = nn.Linear(128, num_classes)
        
    def forward(self, batch_data):
        token_embs = self.emb(batch_data)
        outputs, (h_n, c_n) = self.lstm(token_embs)
        
        last_hidden_state = h_n
        last_hidden_state = last_hidden_state.permute(1, 0, 2)
        last_hidden_state = last_hidden_state.flatten(start_dim = 1)

        last_hidden_state = self.relu(last_hidden_state)
        logits = self.lin(last_hidden_state)
        
        return logits
        

In [111]:
model = NLP_model(vocab = dataset.vocab, emb_size = 50, num_classes = 3)
model

NLP_model(
  (emb): Embedding(1183, 50)
  (lstm): LSTM(50, 32, num_layers=2, batch_first=True, bidirectional=True)
  (relu): ReLU()
  (lin): Linear(in_features=128, out_features=3, bias=True)
)

In [112]:
from torch import optim
opt = optim.Adam(model.parameters(), lr=0.001)
opt

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [113]:
loss_func = nn.CrossEntropyLoss(torch.tensor([4.61, 1, 5]))
# loss_func = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=0.001)

In [114]:
def calc_acc(preds, batched_labels):
    predicted_classes = torch.softmax(preds, dim = 1).argmax(dim = 1)

    num_correct = (predicted_classes == batched_labels).sum()

    acc = num_correct/len(batched_labels)
    return acc

In [115]:
def calc_classwise_acc(preds, batched_labels):
    predicted_classes = torch.softmax(preds, dim = 1).argmax(dim = 1)
    pred_0 = (predicted_classes == 0)
    pred_1 = (predicted_classes == 1)
    pred_2 = (predicted_classes == 2)
    label_0 = (batched_labels == 0)
    label_1 = (batched_labels == 1)
    label_2 = (batched_labels == 2)
    num_correct_0 = ((pred_0.long() + label_0.long())==2).sum()
    num_correct_1 = ((pred_1.long() + label_1.long())==2).sum()
    num_correct_2 = ((pred_2.long() + label_2.long())==2).sum()
    num_0 = label_0.sum()
    num_1 = label_1.sum()
    num_2 = label_2.sum()
    if(num_0 == 0):
        acc_0 = 1
    else:
        acc_0 = num_correct_0 / num_0
    if(num_1 == 0):
        acc_1 = 1
    else:
        acc_1 = num_correct_1 / num_1
    if(num_2 == 0):
        acc_2 = 1
    else:
        acc_2 = num_correct_2 / num_2
    

    return acc_0,acc_1,acc_2

In [116]:
print_rate = 50
num_epochs = 5
for epoch in range(num_epochs):
    print("-" * 50)
    total_loss = 0.0 
    total_acc = 0.0
    total_acc_0 = 0
    total_acc_1 = 0
    total_acc_2 = 0
    model.train()
    for i, (batched_data, batched_labels) in enumerate(train_dataloader,start=1):
        preds = model(batched_data)
        loss = loss_func(preds, batched_labels)
        total_loss+=loss
        accuracy = calc_acc(preds, batched_labels)
        total_acc+=accuracy
        acc_0, acc_1, acc_2 = calc_classwise_acc(preds, batched_labels)
        total_acc_0+=acc_0
        total_acc_1+=acc_1
        total_acc_2+=acc_2
        if (i % print_rate == 0):
            print("Train: loss : {0}, accuracy: {1} | acc_0: {2}, acc_1: {3}, acc_2: {4}".format(total_loss/print_rate, total_acc/print_rate, total_acc_0/print_rate,total_acc_1/print_rate,total_acc_2/print_rate))
            total_loss=0.0
            total_acc=0.0
            total_acc_0 = 0
            total_acc_1 = 0
            total_acc_2 = 0
        loss.backward()
        opt.step()
        opt.zero_grad()
    total_loss=0.0
    total_acc=0.0
    total_acc_0 = 0
    total_acc_1 = 0
    total_acc_2 = 0
    model.eval()
    for i, (batched_data, batched_labels) in enumerate(test_dataloader):
        # print(i)
        with torch.no_grad():
            preds = model(batched_data)
            loss = loss_func(preds, batched_labels)
            total_loss+=loss
            accuracy = calc_acc(preds, batched_labels)
            total_acc+=accuracy
            acc_0, acc_1, acc_2 = calc_classwise_acc(preds, batched_labels)
            total_acc_0+=acc_0
            total_acc_1+=acc_1
            total_acc_2+=acc_2
    print("Test: loss : {0}, accuracy: {1} | acc_0: {2}, acc_1: {3}, acc_2: {4}".format(total_loss/len(test_dataloader), total_acc/len(test_dataloader), total_acc_0/len(test_dataloader),total_acc_1/len(test_dataloader),total_acc_2/len(test_dataloader)))

--------------------------------------------------
Train: loss : 1.0241116285324097, accuracy: 0.5137500166893005 | acc_0: 0.11999999731779099, acc_1: 0.5487442016601562, acc_2: 0.45916667580604553
Train: loss : 0.9892950654029846, accuracy: 0.5143749713897705 | acc_0: 0.2199999988079071, acc_1: 0.5534712076187134, acc_2: 0.545175313949585
Train: loss : 0.9747843146324158, accuracy: 0.5762500166893005 | acc_0: 0.18000000715255737, acc_1: 0.5936287641525269, acc_2: 0.6985634565353394
Train: loss : 0.8745886087417603, accuracy: 0.6631249785423279 | acc_0: 0.03999999910593033, acc_1: 0.6825560927391052, acc_2: 0.8402445316314697
Train: loss : 0.7593048214912415, accuracy: 0.7331249713897705 | acc_0: 0.20000000298023224, acc_1: 0.7575266361236572, acc_2: 0.8937143087387085
Train: loss : 0.6878086924552917, accuracy: 0.7681249976158142 | acc_0: 0.18000000715255737, acc_1: 0.7924449443817139, acc_2: 0.9007936716079712
Train: loss : 0.6566739082336426, accuracy: 0.8012499809265137 | acc_0: 0.

In [117]:
params = model.state_dict()
torch.save(params, 'my_model_weights.pt')

In [118]:
params_loaded = torch.load('my_model_weights.pt')
model.load_state_dict(params_loaded)

NameError: name 'model2' is not defined